<a href="https://colab.research.google.com/github/geekzenpro/material-dashboard/blob/master/sp_exo_local_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import time
import requests
import traceback
from pandas.tseries.offsets import BDay
import numpy as np
from pycaret.datasets import get_data
from pycaret.time_series import TSForecastingExperiment
import os
import pickle
global_fig_settings = {
    # "renderer": "notebook",
    "renderer": "png",
    "width": 1000,
    "height": 600,
}

def save_columns(df, filename):
    """Save the columns of a DataFrame to a file"""
    with open(filename, 'wb') as f:
        pickle.dump(df.columns.tolist(), f)

def load_columns(filename):
    """Load the columns of a DataFrame from a file"""
    with open(filename, 'rb') as f:
        return pickle.load(f)


def calc_next_friday(d):
    d = datetime.strptime(d, '%Y-%m-%d')
    d += timedelta(7)
    while d.weekday() != 4:
        d += timedelta(1)
    return d

# Function to calculate the expected move using options data
def calculate_expected_move_from_polygonio(symbol, exp_date, close_price):

    api_key='JvOlZnErKIgvJ5yECAnUK6OFBoeKTioJ'

    last_friday_price = close_price
    last_friday = exp_date
    #contractExpirydate = datetime.strptime(exp_date, '%Y-%m-%d').strftime('%y%m%d')
    calculated_Next_Friday = calc_next_friday(exp_date)
    print("calculated_Next_Friday: ", calculated_Next_Friday)
    contractExpirydate =  calculated_Next_Friday.strftime('%y%m%d')
    print("contractExpirydate: ", contractExpirydate)
    #first_value = last_friday_price.iloc[0]
    print("last_friday_price: ", last_friday_price)
    if (last_friday_price < 100).all():
        polygonCallContractName = "O:"+symbol+contractExpirydate+"P"+ "000"+str(int(last_friday_price))+ "000"
    elif last_friday_price >= 1000 and last_friday_price < 10000:
        polygonCallContractName = "O:"+symbol+contractExpirydate+"P"+ "0"+str(int(last_friday_price))+ "000"
    else:
        polygonCallContractName = "O:"+symbol+contractExpirydate+"P"+ "00"+str(int(last_friday_price))+ "000"

    #frame polygon put contract name:
    if last_friday_price < 100:
        polygonPutContractName = "O:"+symbol+contractExpirydate+"C"+ "000"+str(int(last_friday_price))+ "000"
    elif last_friday_price >= 1000 and last_friday_price < 10000:
        polygonPutContractName = "O:"+symbol+contractExpirydate+"C"+ "0"+str(int(last_friday_price))+ "000"
    else:
        polygonPutContractName = "O:"+symbol+contractExpirydate+"C"+ "00"+str(int(last_friday_price))+ "000"

    #print(polygonCallContractName)
    #print(polygonPutContractName)
    try:
        #get the options call data from polygon.io
        url = f'https://api.polygon.io/v2/aggs/ticker/{polygonCallContractName}/range/1/day/{last_friday}/{last_friday}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}'
        #print(url)
        options_data = requests.get(url).json()
        #print(options_data)
        atm_call_close_price = options_data['results'][0]['c']
        #print(polygonCallContractName, atm_call_close_price )
    except Exception as e:
        print(f"Failed to get options data for call: {e}")
        # print("Traceback: \n")
        # traceback.print_exc()  # This will print detailed traceback
        return None

    try:
        #get the options put data from polygon.io
        url = f'https://api.polygon.io/v2/aggs/ticker/{polygonPutContractName}/range/1/day/{last_friday}/{last_friday}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}'
        #print(url)
        options_data = requests.get(url).json()
        #print(options_data)
        atm_put_close_price = options_data['results'][0]['c']
        #print(polygonPutContractName, atm_put_close_price)
    except Exception as e:
        print(f"Failed to get options data for put: {e}")
        # print("Traceback: \n")
        # traceback.print_exc()  # This will print detailed traceback
        return None


    try:

        #calculate the expected move
        #expected_move_friday = (atm_call_close_price + atm_put_close_price) * math.sqrt(7 / 365)  # 7 days to expiration
        print("ATM Call Close Price: ", atm_call_close_price)
        print("ATM Put Close Price: ", atm_put_close_price)

        expected_move_friday = ((atm_call_close_price + atm_put_close_price)/(close_price))*100
        print("Expected_Move_Friday",expected_move_friday)

        stock_price = last_friday_price

        expected_high_friday = stock_price + expected_move_friday
        expected_low_friday = stock_price - expected_move_friday

        #return polygonCallContractName, polygonPutContractName, round(expected_low_friday, 2), round(expected_high_friday, 2), round(last_friday_price, 2), round(expected_move_friday, 2)
        #time.sleep(2)
        return round(expected_move_friday, 2)
    except Exception as e:
        print(f"Failed to calculate expected move: {e}")
        return None

def calculate_ATM_CP_Price_from_polygonio(symbol, exp_date, close_price):

    api_key='JvOlZnErKIgvJ5yECAnUK6OFBoeKTioJ'

    last_friday_price = close_price
    last_friday = exp_date
    #contractExpirydate = datetime.strptime(exp_date, '%Y-%m-%d').strftime('%y%m%d')
    calculated_Next_Friday = calc_next_friday(exp_date)
    print("calculated_Next_Friday: ", calculated_Next_Friday)
    contractExpirydate =  calculated_Next_Friday.strftime('%y%m%d')
    print("contractExpirydate: ", contractExpirydate)
    #first_value = last_friday_price.iloc[0]
    print("last_friday_price: ", last_friday_price)
    if (last_friday_price < 100).all():
        polygonCallContractName = "O:"+symbol+contractExpirydate+"P"+ "000"+str(int(last_friday_price))+ "000"
    elif last_friday_price >= 1000 and last_friday_price < 10000:
        polygonCallContractName = "O:"+symbol+contractExpirydate+"P"+ "0"+str(int(last_friday_price))+ "000"
    else:
        polygonCallContractName = "O:"+symbol+contractExpirydate+"P"+ "00"+str(int(last_friday_price))+ "000"

    #frame polygon put contract name:
    if last_friday_price < 100:
        polygonPutContractName = "O:"+symbol+contractExpirydate+"C"+ "000"+str(int(last_friday_price))+ "000"
    elif last_friday_price >= 1000 and last_friday_price < 10000:
        polygonPutContractName = "O:"+symbol+contractExpirydate+"C"+ "0"+str(int(last_friday_price))+ "000"
    else:
        polygonPutContractName = "O:"+symbol+contractExpirydate+"C"+ "00"+str(int(last_friday_price))+ "000"

    #print(polygonCallContractName)
    #print(polygonPutContractName)
    try:
        #get the options call data from polygon.io
        url = f'https://api.polygon.io/v2/aggs/ticker/{polygonCallContractName}/range/1/day/{last_friday}/{last_friday}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}'
        #print(url)
        options_data = requests.get(url).json()
        #print(options_data)
        atm_call_close_price = options_data['results'][0]['c']
        #print(polygonCallContractName, atm_call_close_price )
    except Exception as e:
        print(f"Failed to get options data for call: {e}")
        # print("Traceback: \n")
        # traceback.print_exc()  # This will print detailed traceback
        atm_call_close_price = 0

    try:
        #get the options put data from polygon.io
        url = f'https://api.polygon.io/v2/aggs/ticker/{polygonPutContractName}/range/1/day/{last_friday}/{last_friday}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}'
        #print(url)
        options_data = requests.get(url).json()
        #print(options_data)
        atm_put_close_price = options_data['results'][0]['c']
        #print(polygonPutContractName, atm_put_close_price)
    except Exception as e:
        print(f"Failed to get options data for put: {e}")
        # print("Traceback: \n")
        # traceback.print_exc()  # This will print detailed traceback
        atm_call_close_price = 0


    try:

        #calculate the expected move
        #expected_move_friday = (atm_call_close_price + atm_put_close_price) * math.sqrt(7 / 365)  # 7 days to expiration
        print("ATM Call Close Price: ", atm_call_close_price)
        print("ATM Put Close Price: ", atm_put_close_price)


        return round(atm_call_close_price,2), round(atm_put_close_price,2)
    except Exception as e:
        print(f"Failed to calculate ATM PC Price: {e}")
        return 0, 0

def calculate_returns_for_dates(pull_start_date, pull_end_date, tickers, EM_tickers):
    returns = pd.DataFrame()
    data = pd.DataFrame()
    # Create an empty DataFrame for storing the returns
    print("Pull Start Date: ", pull_start_date)
    print("Pull End Date: ", pull_end_date)
    # Download the historical data and calculate returns for each ticker
    for ticker in tickers:
        data = yf.download(ticker, start=pull_start_date, end=pull_end_date)
    #   close = data['Adj Close']
    #   open = data['Open']
    #  #temp additions of open & close for comaprisions
    #   ret = open.rename(ticker + "_open")
    #   returns = pd.concat([returns, ret], axis=1)
    #   ret = close.rename(ticker + "_close")
    #   returns = pd.concat([returns, ret], axis=1)

        #Quant Returns
        close = data['Adj Close'].shift(-1)
        open = data['Open']
        ret = ((open - close) / close).fillna(0).rename(ticker +"_qreturns")
        #ret = close.pct_change().rename(ticker)
        # round of the returns to 4 decimal places
        ret = round(ret*100, 2)
        returns = pd.concat([returns, ret], axis=1)

        close = data['Adj Close']
        #open returns for the next day
        open = data['Open']

        ret = open.pct_change().shift(-1).rename(ticker +"_oreturns")
        #ret = close.pct_change().rename(ticker)
        # round of the returns to 4 decimal places
        ret = round(ret*100, 2)
        returns = pd.concat([returns, ret], axis=1)

        #close returns for the next day
        ret = close.pct_change().shift(-1).rename(ticker +"_creturns")
        #ret = close.pct_change().rename(ticker)
        # round of the returns to 4 decimal places
        ret = round(ret*100, 2)
        returns = pd.concat([returns, ret], axis=1)

        #high returns for the next day
        high = data['High']
        ret = ((close - high) / close).rename(ticker +"_hreturns")
        ret = round(ret*100, 2)
        returns = pd.concat([returns, ret], axis=1)

        #low returns for the next day
        low = data['Low']
        ret = ((close - low) / close).rename(ticker +"_lreturns")
        ret = round(ret*100, 2)
        returns = pd.concat([returns, ret], axis=1)

        #capture daily open to close returns
        ret = ((close - open) / open).rename(ticker + "_ocreturns")
        ret = round(ret*100, 2)
        returns = pd.concat([returns, ret], axis=1)

        data[ticker + '_close'] = round(data['Close'],2)
        returns = pd.concat([returns, data[ticker + '_close']], axis=1)

        new_data = pd.DataFrame(index=returns.index)
        if ticker in EM_tickers:
            # Start with an empty dictionary
            expected_moves = {}
            polygonCount=0
            EMCallDataIssue=False
            # Iterate through each date in the returns DataFrame

            for date in returns.index:
                # Calculate the last Friday's date for the current date
                last_friday = date - timedelta(days=(date.weekday() - 4) % 7)
                #print(date, last_friday)
                # Format the date as a string for the API call
                last_friday_str = last_friday.strftime('%Y-%m-%d')
                print("Price close Date:", date, "Last Friday Date",last_friday_str)
                # Check if the expected move for the last Friday already exists in the dictionary
                #print(returns)
                #print(returns.loc[date, ticker + '_close'])
                if last_friday_str not in expected_moves:
                    EMCallDataIssue=False
                    # If it doesn't exist, calculate the expected move and store it in the dictionary
                    #print("EM for this ", last_friday_str, " Not Available", ticker, returns.loc[date, ticker + '_close'])
                    exected_move_as_of_friday = calculate_expected_move_from_polygonio(ticker, last_friday_str, returns.loc[date, ticker + '_close'])
                    #time.sleep(31)
                    polygonCount=polygonCount+1
                    #print(polygonCount)
                    if exected_move_as_of_friday is None:
                    # Calculate the date of the previous Friday
                        prev_last_friday = date - timedelta(days=(date.weekday() - 4) % 7 + 7)
                        prev_last_friday_str = prev_last_friday.strftime('%Y-%m-%d')
                        # Get the expected move from the previous Friday from the dictionary
                        if prev_last_friday_str in expected_moves:
                            #print("pulling data from previous last Friday, previous last Friday is ", prev_last_friday_str)
                            # print(expected_moves)
                            expected_moves[last_friday_str] = expected_moves[prev_last_friday_str]
                            expected_move = expected_moves[prev_last_friday_str]
                            EMCallDataIssue=True
                        else:
                            #print("Printing all Expected Moves calculated so far")
                            # print(expected_moves)
                            # print(f"No expected move available for date {prev_last_friday_str}")
                            #Assume a 2% expected move if no data is available previously as well
                            expected_moves[last_friday_str] = returns.loc[date, ticker + '_close'] * 0.02
                            expected_move = expected_moves[last_friday_str]
                            EMCallDataIssue=True
                    else:
                        expected_moves[last_friday_str] = exected_move_as_of_friday
                        expected_move = exected_move_as_of_friday
                        EMCallDataIssue=False
                        #print("No Calculation Issue for ", last_friday_str, " Expected Move is ", expected_move)

                # Retrieve the expected move for last Friday
                print("Expected Move", ticker, last_friday_str, expected_moves[last_friday_str])
                expected_move = expected_moves[last_friday_str]
                # Calculate expected_move_percent
                #expected_move_percent = expected_move / returns.loc[date, ticker + '_close']
                #check if returns.loc[last_friday, ticker + '_close']  exists or not

                if last_friday in returns.index:
                    expected_move_percent =  (returns.loc[date, ticker + '_close'] - returns.loc[last_friday, ticker + '_close'] )/expected_move
                    # print ("Last Friday Close", returns.loc[last_friday, ticker + '_close'])
                    # print ("Today Close", returns.loc[date, ticker + '_close'])
                    # print ("Expected Move ", expected_move)
                    # print ("Expected Move Percent", expected_move_percent)
                    lower_bound = returns.loc[last_friday, ticker + '_close'] - expected_move
                    upper_bound = returns.loc[last_friday, ticker + '_close'] + expected_move
                    if returns.loc[date, ticker + '_close'] < lower_bound:
                        expected_move_range = -1
                    elif returns.loc[date, ticker + '_close'] > upper_bound:
                        expected_move_range = 1
                    else:
                        expected_move_range = 0
                else:
                    expected_move_percent =  0
                    expected_move_range = 0
                # Calculate days_to_expire
                days_to_expire = len(pd.bdate_range(date, last_friday + pd.DateOffset(weeks=1))) - 1
                #calculate the ATM Call & Put Price for the day
                atm_call_price, atm_put_price = calculate_ATM_CP_Price_from_polygonio(ticker, last_friday_str, returns.loc[date, ticker + '_close'])
                # if EMCallDataIssue is True:
                #      new_data.loc[date, ticker + '_EMCalcIssue'] = "True - PreviousWeekEMisUsed"
                # else:
                #      new_data.loc[date, ticker + '_EMCalcIssue'] = "False - No Issue"
                new_data.loc[date, ticker + '_EM'] = expected_move
                new_data.loc[date, ticker + '_AtmCallPrice'] = atm_call_price
                new_data.loc[date, ticker + '_AtmPutPrice'] = atm_put_price
                new_data.loc[date, ticker + '_Pct_of_EM'] = round(expected_move_percent,4)
                new_data.loc[date, ticker + '_EM_DTE'] = days_to_expire
                new_data.loc[date, ticker + '_EM_BR'] = expected_move_range
                # Concatenate the two DataFrames along axis=1
            returns = pd.concat([returns, new_data], axis=1)
    return returns

def ExtractAndPrepData(start_date, end_date, tickers,EM_tickers, cached_returns_file, target, tradeday):

    """
    Function to Extract and Prepare Data.
    """
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    tradeday = pd.to_datetime(tradeday)

    root_dir = os.path.dirname(os.path.abspath(__file__))
    exo_data_file_path = os.path.join(root_dir, 'exo-data-files', cached_returns_file)
    cached_returns_file_w_path = os.path.join(root_dir, exo_data_file_path)

    root_dir = os.path.dirname(os.path.abspath(__file__))
    model_name = "exo_model_target-" + target + str(forcast_days_fh)
    modelColumnsFileName = model_name + "_columns.pkl"
    modelColumnsFileName_with_path = os.path.join(root_dir, 'exo-model-attributes', modelColumnsFileName)

    print(cached_returns_file_w_path)
    #cached_returns_file_w_path = '/Users/zenpro/src/mqc7/lchatgpt/quantgalore/exoquant/AMZN_ARIMAX_cached_returns_data_file.csv'
    #print(os.path)
    returnsAppeneded = False
    #returns = pd.read_csv(cached_returns_file, index_col=0)
    if os.path.exists(cached_returns_file_w_path):
        returns = pd.read_csv(cached_returns_file_w_path)
        if 'Unnamed: 0' in returns.columns:
            returns = returns.drop(columns=['Unnamed: 0'])
        returns.set_index('Date', inplace=True)
        returns.index = pd.to_datetime(returns.index)
        print(returns)
        returns = returns[~returns.index.duplicated(keep='first')]
        print("deduplicate", returns)
        #returns['Date'] = returns.index.date
        print(returns.head())
    else:
        returns = pd.DataFrame()
    #print(returns.head())
    if not returns.empty:
        min_cached_date = returns.index.min()
        max_cached_date = returns.index.max()
        print("cache min & max ", min_cached_date, max_cached_date)
        print("start & end date ",start_date, end_date)
        if max_cached_date < end_date:
            print("inside less")

            print("max_cached_date", max_cached_date)
            print("end_date", end_date)

            missing_returns_end =   calculate_returns_for_dates(max_cached_date , end_date+ pd.Timedelta(days=1), tickers, EM_tickers)
            print(missing_returns_end)
            returns = pd.concat([returns, missing_returns_end])
            returnsAppeneded = True

        if min_cached_date > start_date:
            print("greater than")
            print("min_cached_date", min_cached_date)
            print("start_date", start_date)
            missing_returns_start =  calculate_returns_for_dates(start_date- pd.Timedelta(days=1), min_cached_date , tickers, EM_tickers)
            print(missing_returns_start)
            returns = pd.concat([missing_returns_start, returns])
            returnsAppeneded = True
    else:
        print("returns file is empty")
        returns = calculate_returns_for_dates(start_date,end_date, tickers, EM_tickers)
        returns_sc = returns.copy()
        print(returns_sc.head())
        returns_sc.index = pd.to_datetime(returns_sc.index) # make sure the index is a DatetimeIndex
        returns_sc['Date'] = returns_sc.index.date
        returns_sc = returns_sc[returns_sc.index < tradeday]
        returns_sc.reset_index(drop=True, inplace=True)
        returns_sc = returns_sc.loc[~returns_sc.index.duplicated(keep='first')]
        returns_sc.replace([np.inf, -np.inf], 0, inplace=True)
        returns_sc.fillna(0, inplace=True)
        # returns_sc = returns_sc.iloc[:-1]
        returns_sc.to_csv(cached_returns_file_w_path)
        print("New returns file is created")
    #returns.sort_index(inplace=True)
    if (returnsAppeneded == True):
        returns_sca = returns.copy()
        returns_sca.index = pd.to_datetime(returns_sca.index) # make sure the index is a DatetimeIndex
        returns_sca['Date'] = returns_sca.index.date
        returns_sca = returns_sca[returns_sca.index < tradeday]
        returns_sca.reset_index(drop=True, inplace=True)
        returns_sca = returns_sca.loc[~returns_sca.index.duplicated(keep='first')]
        returns_sca.replace([np.inf, -np.inf], 0, inplace=True)
        returns_sca.fillna(0, inplace=True)
        returns_sca.to_csv(cached_returns_file_w_path)
    # restrict to desired date range
    returns = returns.loc[start_date:end_date]
    returns = returns[~returns.index.duplicated(keep='first')]
    #return returns
    # #today = datetime.today()+ timedelta(days=1)
    # today = datetime.today()+ timedelta(days=-2)
    # predict_end_date =today.strftime('%Y-%m-%d')
    # #start date is 180 trading days before the end date
    # #predict_start_date = datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=2)
    # predict_start_date = datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=5)
    # print("predict_start_date", predict_start_date)
    # print("predict_end_date", predict_end_date)
    # Ensure the index is a DatetimeIndex
    returns.index = pd.to_datetime(returns.index)

    # Create a mask for dates prior to today
    mask = returns.index.date < tradeday.date()

    # Apply the mask to the DataFrame
    returns = returns.loc[mask]

    # Add a 'Date' column based on the index
    returns['Date'] = returns.index.date

    # Reset the index
    returns.reset_index(drop=True, inplace=True)
    returns["index"] = pd.to_datetime(returns["Date"])
    returns.drop(columns=["Date"], inplace=True)
    #returns.replace(-200, np.nan, inplace=True)
    #save the returns file only if there is any changes to the file


    #use the tickers to create the list of new fields  that ends with _close
    exclude = [ticker + '_close' for ticker in tickers]
    #print(exclude)
    data_exo = returns.copy()
    data_exo.drop(columns=exclude, inplace=True)
    data_exo.replace([np.inf, -np.inf], 0, inplace=True)
    data_exo = data_exo.loc[~data_exo.index.duplicated(keep='first')]
    print("printing duplicate data_exo index")
    print(data_exo.index.duplicated().sum())

    #data_exo = data_exo.loc[~returns.index.duplicated(keep='first')]
    # Remove rows with all NaN values
    data_exo.dropna(how='all', inplace=True)

    # If there are still NaN values, you can fill them with an appropriate value, such as 0
    data_exo.fillna(0, inplace=True)

    # Make sure there are no duplicates
    assert data_exo.index.duplicated().sum() == 0
    data_exo.set_index("index", inplace=True)
    data_exo = data_exo.asfreq('B')
    data_exo.head()

    save_columns(data_exo, modelColumnsFileName_with_path)
    #print(type(data_exo))
    #print(data_exo.index)
    #data_exo.to_csv(target+"exo_data.csv")
    return data_exo

def createFinalizeSaveModel(data_input, target, forcast_days_fh):
    exp_exo = TSForecastingExperiment()
    #print(data_input)
    print(type(data_input))
    #print(data_input.index)
    pd.set_option('display.max_columns', None)
    #print(data_input)
    print("length of exog_var: ", len(data_input.columns))
    print("length of exog_var: ", len(data_input))
    max_day_trained_data = data_input.index.max()
    print("max_day_trained_data", max_day_trained_data)
    exp_exo.setup(
        data=data_input, target=target, fh=forcast_days_fh,
        numeric_imputation_target="ffill", numeric_imputation_exogenous="ffill",
        fig_kwargs=global_fig_settings, session_id=12
    )
    model_exo = exp_exo.create_model("auto_arima")
    # data_exo.plot()
    # exp_exo.plot_model(model_exo)
    #tuned_model_exo = exp.tune_model(model_exo)

    final_exo_model = exp_exo.finalize_model(model_exo)
    root_dir = os.path.dirname(os.path.abspath(__file__))
    model_name = "exo_model_target-" + target + str(forcast_days_fh)
    models_FilePath = os.path.join(root_dir, 'exo-models', model_name)
    _ = exp_exo.save_model(final_exo_model, models_FilePath)

def prediction(target, tickers, EM_tickers, predict_start_date, predict_end_date, forcast_days_fh):
    td_returns = calculate_returns_for_dates(predict_start_date,predict_end_date, tickers, EM_tickers)
    # Ensure the index is a DatetimeIndex
    td_returns.index = pd.to_datetime(td_returns.index)
    # # Create a mask for dates prior to today
    # mask = returns.index.date >= datetime.today().date()
    # # Apply the mask to the DataFrame
    # returns = returns.loc[mask]

    # Add a 'Date' column based on the index
    exclude = [target]
    td_returns.drop(columns=exclude, inplace=True)

    td_returns['Date'] = td_returns.index.date

    # Reset the index
    td_returns.reset_index(drop=True, inplace=True)

    td_returns["index"] = pd.to_datetime(td_returns["Date"])
    td_returns.drop(columns=["Date"], inplace=True)
    #td_returns.head()
    #get close price from the last day of the tdReturns dataframe
    #print(td_returns.tail(1))
    #print(td_returns.tail(1).index)
    td_max_date = td_returns.tail(1).index
    td_close_price = td_returns.tail(1)['SPY_close']
    print("td_max_date", td_max_date)
    print("td_close_price", td_close_price)
    print(td_returns)

    td_returns.iloc[1] = td_returns.iloc[1].fillna(td_returns.iloc[0])
    # Reset the index
    ftd_returns = td_returns.drop(td_returns.index[0])
    # exclude = ['XLK_close', 'XLF_close','XLI_close', 'XLE_close' ,'XLY_close','XLP_close','XLU_close','XLV_close','XLB_close', 'XLRE_close','XLC_close','SPY_close','QQQ_close','VXX_close', 'UVXY_close','^VIX_close', '^VIX9D_close', '^VIX3M_close']

    # ftd_returns.drop(columns=exclude, inplace=True)
    ftd_returns.set_index("index", inplace=True)
    #data_exo.reindex(schedule.index)
    ftd_returns = ftd_returns.asfreq('B')
    ftd_returns.index = ftd_returns.index.to_period('B')
    pd.set_option('display.max_columns', None)
    #print(ftd_returns)
    exp_future = TSForecastingExperiment()

    model_name = "exo_model_target-" + target + str(forcast_days_fh)
    root_dir = os.path.dirname(os.path.abspath(__file__))

    modelColumnFileName = model_name + "_columns.pkl"
    modelColumnsFileName_with_path = os.path.join(root_dir, 'exo-model-attributes', modelColumnFileName)

    root_dir = os.path.dirname(os.path.abspath(__file__))
    model_name = "exo_model_target-" + target + str(forcast_days_fh)
    model_FilePath = os.path.join(root_dir, 'exo-models', model_name)
    modelColumnsFileName = model_name + "_columns.pkl"
    modelColumnsFileName_with_path = os.path.join(root_dir, 'exo-model-attributes', modelColumnsFileName)


    final_exo_model = exp_future.load_model(model_FilePath)
    columns_used_in_training = load_columns(modelColumnsFileName_with_path)

    #print("columns_used_in_training", columns_used_in_training)
    print("length of columns_used_in_training", len(columns_used_in_training))
    columns_used_in_training.remove(target)
    ftd_returns_exog = ftd_returns[columns_used_in_training]
    ftd_returns_exog.replace([np.inf, -np.inf], np.nan, inplace=True)
    ftd_returns_exog.fillna(0, inplace=True)
    #print(ftd_returns_exog)
    ftd_returns_exog_copy = ftd_returns_exog.copy()
    pd.set_option('display.max_columns', None)
    print("length of exog_var: ", len(ftd_returns_exog.columns))
    #print("index date in exog_var dataframe: ", ftd_returns_exog.index)
    #print("exog_var:\n", ftd_returns_exog_copy)
    future_preds = exp_future.predict_model(final_exo_model, X=ftd_returns_exog)
    return future_preds, td_close_price, td_max_date

#end date is the current date
# today = datetime.today()+ timedelta(days=-1)
# end_date =today.strftime('%Y-%m-%d')
# #start date is 180 trading days before the end date
# start_date = datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=29)
# print("start_date: ", start_date)
# print("end_date: ", end_date)

#predict_start_date = datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=5)
ticker = 'SPY'
forcast_days_fh = 1
EM_tickers = [ticker,'QQQ']
target = ticker + "_ocreturns"
tradeday = datetime.today().date()

# If yesterday was a weekend, adjust 'today' to be the last trading day
if tradeday.weekday() > 4:  # 0-4 corresponds to Monday-Friday
    # Subtract the number of days to the last trading day (Friday)
    tradeday = tradeday - timedelta(days=tradeday.weekday() - 4)

# Convert 'today' back to a datetime object with time (at midnight)
tradeday = pd.to_datetime(tradeday) #- timedelta(days=1)
print("tradeday: ", tradeday)
#tradeday = '2023-07-14'

end_date = (tradeday+ timedelta(days=1)).strftime('%Y-%m-%d')
start_date = datetime.strptime(end_date, '%Y-%m-%d') + timedelta(days=-290)
start_date = start_date.strftime('%Y-%m-%d')
print("start_date: ", start_date)
print("end_date: ", end_date)
predict_end_date =(tradeday + timedelta(days=1)).strftime('%Y-%m-%d')
predict_start_date = datetime.strptime(predict_end_date, '%Y-%m-%d') - timedelta(days=2)
predict_start_date = predict_start_date.strftime('%Y-%m-%d')
tickers = [ticker, 'XLK', 'XLF','XLI', 'XLE' ,'XLY','XLP','XLU','XLV','XLB', 'XLRE','XLC','QQQ','VXX', 'UVXY','^VIX', '^VIX9D', '^VIX3M']
cache_file_name = ticker + '_ARIMAX_cached_returns_data_file_enhanced.csv'
data_exo = ExtractAndPrepData(start_date, end_date, tickers, EM_tickers, cache_file_name, target, tradeday)
createFinalizeSaveModel(data_exo, target, forcast_days_fh)
print("predict_start_date", predict_start_date)
print("predict_end_date", predict_end_date)
predResults , td_close_price, td_max_date= prediction(target, tickers, EM_tickers, predict_start_date, predict_end_date, forcast_days_fh)
# get date and y_pred from predResults
print("predResults: ", predResults)
pd_date = predResults.index
pd_Y_pred = predResults['y_pred']
print("current price for " + ticker +":", td_close_price)
#get current price of SPX close price from yahoo finance

if target == ticker + "_ocreturns":
    spy_close_price = round(yf.download(ticker, start=tradeday, end=tradeday + pd.Timedelta(days=1))['Open'][0],2)

    print(spy_close_price)
    #print( target, + " for " + ticker + " on " + str(td_max_date) + " is " + str(td_close_price * (1 + pd_Y_pred)))
    print(f"{target} for {ticker} on {str(td_max_date)} is {str(spy_close_price * (1 + (pd_Y_pred/100)))}")


    spx_close_price = round(yf.download("^SPX", start=tradeday, end=tradeday + pd.Timedelta(days=1))['Open'][0],2)

    print("current price: ", td_close_price)
    print(f"{target} for SPX on {str(td_max_date)} is {str(spx_close_price * (1 + (pd_Y_pred/100)))}")


if target == ticker + "_qreturns":
    spy_close_price = round(yf.download(ticker, start=tradeday, end=tradeday + pd.Timedelta(days=1))['Close'][0],2)

    print(spy_close_price)
    #print( target, + " for " + ticker + " on " + str(td_max_date) + " is " + str(td_close_price * (1 + pd_Y_pred)))
    print(f"{target} for {ticker} on {str(td_max_date)} is {str(spy_close_price * (1 + (pd_Y_pred/100)))}")

    spx_close_price = round(yf.download("^SPX", start=tradeday, end=tradeday + pd.Timedelta(days=1))['Close'][0],2)

    print("current price: ", td_close_price)
    print(f"{target} for SPX on {str(td_max_date)} is {str(spx_close_price * (1 + (pd_Y_pred/100)))}")


# spx_close_price = round(yf.download("^SPX", start=tradeday, end=tradeday + pd.Timedelta(days=1))['Close'][0],2)

# print("current price: ", td_close_price)
# print(f"{target} for SPX on {str(td_max_date)} is {str(spx_close_price * (1 + (pd_Y_pred/100)))}")
